In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import configparser
from influxdb_client import InfluxDBClient, QueryApi
# import plotly.io as pio; pio.renderers.default = "browser"
import plotly.io as pio; pio.renderers.default = "notebook_connected"
import warnings
import pandas as pd, plotly.express as px
warnings.filterwarnings('ignore')

In [5]:
def run_query(query):
    """웹 API의 runQuery와 동일한 로직"""
    rows = []
    try:
        result = query_api.query(query)
        for table in result:
            for record in table.records:
                o = {}
                for key, value in record.values.items():
                    o[key] = value
                rows.append(o)
    except Exception as e:
        print(f"Query error: {e}")
    return rows

#### InfluxDB 연결 설정

In [58]:
config = configparser.ConfigParser()
config.read('config.ini')

client = InfluxDBClient(
    url=config['influxdb']['url'],
    token=config['influxdb']['token'],
    org=config['influxdb']['org'],
    timeout=1800000  # 30분 타임아웃
)

query_api = client.query_api()
bucket_name = config['influxdb']['bucket']

# 빠른 모드: 동적 조회 제거 (실행 시간 단축)
car_types = ['BONGO3', 'GV60', 'PORTER2']
min_time = '2022-12-01T00:00:00Z'
max_time = '2023-09-01T00:00:00Z'

min_str = '2023-12-01'
max_str = '2023-09-01'
time_range = f"{min_str} ~ {max_str}"

print(f"🔌 InfluxDB 연결 완료: {bucket_name}")
print(f"📅 분석 기간: {time_range}")
print(f"🚗 분석 차종: {', '.join(car_types)}")

🔌 InfluxDB 연결 완료: aicar-bucket
📅 분석 기간: 2023-12-01 ~ 2023-09-01
🚗 분석 차종: BONGO3, GV60, PORTER2


#### 주행 패턴·충전 패턴·시간 기반 분석

##### 전체 라인 수
- 컬럼: 모든 _measurement의 _value 개수
- 계산: count() 집계
- 구현: BMS + GPS 전체 레코드 개수

In [59]:
def get_detailed_records_by_cartype():
    """웹 API getVehicleCounts와 정확히 동일한 로직"""
    car_types = ['BONGO3', 'GV60', 'PORTER2']
    detailed_records = {}
    
    for car_type in car_types:
        bms_query = f'''
        from(bucket: "{bucket_name}")
          |> range(start: {min_time}, stop: {max_time})
          |> filter(fn: (r) => r._measurement == "aicar_bms")
          |> filter(fn: (r) => r.car_type == "{car_type}")
          |> count()
        '''
        
        gps_query = f'''
        from(bucket: "{bucket_name}")
          |> range(start: {min_time}, stop: {max_time})
          |> filter(fn: (r) => r._measurement == "aicar_gps")
          |> filter(fn: (r) => r.car_type == "{car_type}")
          |> count()
        '''
        
        bms_count = 0
        gps_count = 0
        
        try:
            bms_result = run_query(bms_query)
            for record in bms_result:
                bms_count += int(record.get('_value', 0) or 0)
        except Exception as e:
            print(f"BMS count error for {car_type}: {e}")
        
        try:
            gps_result = run_query(gps_query)
            for record in gps_result:
                gps_count += int(record.get('_value', 0) or 0)
        except Exception as e:
            print(f"GPS count error for {car_type}: {e}")
        
        detailed_records[car_type] = {
            'BMS': bms_count,
            'GPS': gps_count,
            '총합': bms_count + gps_count
        }
    
    return detailed_records

def calculate_total_stats(detailed_records):
    """웹 API calculateTotalStats와 정확히 동일한 로직"""
    total_bms = sum(r['BMS'] for r in detailed_records.values())
    total_gps = sum(r['GPS'] for r in detailed_records.values())
    total_sum = total_bms + total_gps
    
    return {'totalBMS': total_bms, 'totalGPS': total_gps, 'totalSum': total_sum}


In [60]:
# 상세 레코드 수 계산
detailed_records = get_detailed_records_by_cartype()

# 결과를 DataFrame으로 변환
records_data = []
for car_type, data in detailed_records.items():
    records_data.append({
        '차종': car_type,
        'BMS': data['BMS'],
        'GPS': data['GPS'],
        '총합': data['총합']
    })

df_records = pd.DataFrame(records_data)

# 테이블 출력
print("📊 차종별 레코드 수 상세 분석:")
print(df_records.to_string(index=False))

# 총합 계산
total_stats = calculate_total_stats(detailed_records)
print(f"\n🔋 BMS 전체 레코드 수: {total_stats['totalBMS']:,}개")
print(f"📍 GPS 전체 레코드 수: {total_stats['totalGPS']:,}개")
print(f"🔢 전체 총 레코드 수: {total_stats['totalSum']:,}개")

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'domain'}, {'type': 'domain'}]],
    subplot_titles=['BMS 레코드 비율', 'GPS 레코드 비율'],
    horizontal_spacing=0.12  # 차트 간 간격
)

# BMS 파이
fig.add_trace(
    go.Pie(labels=df_records['차종'], values=df_records['BMS'], name='BMS',
           textinfo='label+percent', hovertemplate='%{label}<br>%{value:,}'),
    row=1, col=1
)

# GPS 파이
fig.add_trace(
    go.Pie(labels=df_records['차종'], values=df_records['GPS'], name='GPS',
           textinfo='label+percent', hovertemplate='%{label}<br>%{value:,}'),
    row=1, col=2
)

fig.update_layout(
    title='차종별 레코드 수 (BMS vs GPS)',
    showlegend=True,
    width=1000,   # 가로
    height=500    # 세로 (가로:세로 = 2:1)
)

fig.show()

📊 차종별 레코드 수 상세 분석:
     차종        BMS      GPS         총합
 BONGO3 1338356707 75233565 1413590272
   GV60   43442321  2889495   46331816
PORTER2 1391482934 77454905 1468937839

🔋 BMS 전체 레코드 수: 2,773,281,962개
📍 GPS 전체 레코드 수: 155,577,965개
🔢 전체 총 레코드 수: 2,928,859,927개


##### 주행 횟수
- 컬럼: BMS odometer(주행거리계)
- 계산: odometer 증가 구간을 주행으로 간주. 3분 이상 정차 시 주행 종료, 재시작 시 새 주행으로 카운트
- 구현: 시계열 정렬 후 odometer 변화 패턴으로 주행 시작/종료 감지, 3분 임계값 적용

- config.ini의 [driving] 섹션과 파라미터 확인 테스트

In [ ]:
import configparser

config = configparser.ConfigParser()

# BOM 문제를 피하려면 utf-8-sig로 읽기
config.read("config.ini", encoding="utf-8-sig")

print("읽은 섹션:", config.sections()) 

if 'driving' in config:
    IDLE_THRESHOLD_SEC = int(config['driving'].get('idle_threshold_sec', 300))
    MIN_TRIP_DISTANCE = float(config['driving'].get('min_trip_distance', 0.5))
else:
    print("⚠️ [driving] 섹션을 찾을 수 없습니다. 기본값 사용합니다.")
    IDLE_THRESHOLD_SEC = 300
    MIN_TRIP_DISTANCE = 0.5

print("IDLE_THRESHOLD_SEC:", IDLE_THRESHOLD_SEC)
print("MIN_TRIP_DISTANCE:", MIN_TRIP_DISTANCE)

읽은 섹션: ['influxdb', 'driving']
IDLE_THRESHOLD_SEC: 300
MIN_TRIP_DISTANCE: 0.5


- 타입 변환 함수

In [61]:
def auto_convert(value):
    """string 값을 보고 자동으로 int/float 변환"""
    if value is None or value == '':
        return 0
    try:
        if '.' in str(value):
            return float(value)
        else:
            return int(value)
    except:
        return 0

In [ ]:
# 주행 횟수 계산
def get_driving_count_by_cartype():
    """차종별 주행 횟수 계산 (speed > 1.0 연속 구간 기준)"""
    car_types = ['BONGO3', 'GV60', 'PORTER2']
    driving_counts = {}
    
    for car_type in car_types:
        # 주행 중인 데이터만 조회 (speed > 1.0)
        query = f'''
        from(bucket: "{bucket_name}")
          |> range(start: {min_time}, stop: {max_time})
          |> filter(fn: (r) => r._measurement == "aicar_gps")
          |> filter(fn: (r) => r._field == "speed")
          |> filter(fn: (r) => r.car_type == "{car_type}")
          |> keep(columns: ["_time", "_value"])
          |> aggregateWindow(every: 5m, fn: count, createEmpty: false)
        '''
        
        try:
            result = query_api.query(query)
            timestamps = []
            for table in result:
                for record in table.records:
                    speed = auto_convert(record.values.get('_value', 0))
                    if speed > 1.0:
                        timestamps.append(record.get_time())
            
            # 연속 구간 감지 (5분 이상 간격이면 새로운 주행으로 간주)
            driving_count = 0
            if timestamps:
                timestamps.sort()
                last_time = timestamps[0]
                driving_count = 1
                
                for current_time in timestamps[1:]:
                    time_diff = (current_time - last_time).total_seconds()
                    if time_diff > 300:  # 5분 = 300초
                        driving_count += 1
                    last_time = current_time
            
            driving_counts[car_type] = driving_count
            # print(f"✅ {car_type}: {driving_count:,}회 주행")
            
        except Exception as e:
            print(f"❌ {car_type}: 주행 횟수 계산 실패 - {e}")
            driving_counts[car_type] = 0
    
    return driving_counts

In [15]:
# 주행 횟수 계산 실행
driving_counts = get_driving_count_by_cartype()

# 결과를 DataFrame으로 변환
driving_data = []
for car_type, count in driving_counts.items():
    driving_data.append({
        '차종': car_type,
        '주행횟수': count
    })

df_driving = pd.DataFrame(driving_data)

print("\n📊 차종별 주행 횟수 상세 분석:")
print(df_driving.to_string(index=False))

# 총합 계산
total_driving = df_driving['주행횟수'].sum()
print(f"\n🚗 전체 총 주행 횟수: {total_driving:,}회")

# 막대 그래프
fig = px.bar(df_driving, x='차종', y='주행횟수', 
             title='차종별 주행 횟수',
             color='차종',
             text='주행횟수')

fig.update_traces(texttemplate='%{text:,}', textposition='outside')
fig.update_layout(
    yaxis_title='주행 횟수',
    height=500,
    showlegend=False
)
fig.show()

# 파이 차트
fig2 = px.pie(df_driving, values='주행횟수', names='차종',
              title='차종별 주행 횟수 비율')
fig2.update_traces(textposition='inside', textinfo='percent+label')
fig2.show()


📊 차종별 주행 횟수 상세 분석:
     차종  주행횟수
 BONGO3   107
   GV60  5005
PORTER2   169

🚗 전체 총 주행 횟수: 5,281회


##### 평균 SOC, SOH

In [ ]:
def get_avg_soc_soh_by_cartype_webapi_style():
    car_types = ['BONGO3', 'GV60', 'PORTER2']
    avg_data = {}

    for car_type in car_types:
        # 1) SOC/SOH 값 조회
        soc_soh_query = f'''
        from(bucket: "{bucket_name}")
          |> range(start: {min_time}, stop: {max_time})
          |> filter(fn: (r) => r._measurement == "aicar_bms")
          |> filter(fn: (r) => (r._field == "soc" or r._field == "soh"))
          |> filter(fn: (r) => r.car_type == "{car_type}")
          |> filter(fn: (r) => exists r._value)
          |> keep(columns: ["_field", "_value"])
        '''

        # 2) 고유 디바이스 수 조회
        device_query = f'''
        from(bucket: "{bucket_name}")
          |> range(start: {min_time}, stop: {max_time})
          |> filter(fn: (r) => r._measurement == "aicar_bms")
          |> filter(fn: (r) => r.car_type == "{car_type}")
          |> keep(columns: ["device_no"])
          |> distinct(column: "device_no")
        '''

        soc_values, soh_values = [], []

        # 실행
        rows = run_query(soc_soh_query)
        devices = run_query(device_query)

        # 값 수집(0~100만 유효)
        for row in rows:
            val = row.get('_value')
            try:
                v = float(val)
            except:
                continue
            if 0 <= v <= 100:
                if row.get('_field') == 'soc':
                    soc_values.append(v)
                elif row.get('_field') == 'soh':
                    soh_values.append(v)

        avg_soc = sum(soc_values)/len(soc_values) if soc_values else 0.0
        avg_soh = sum(soh_values)/len(soh_values) if soh_values else 0.0
        device_count = len(devices)

        avg_data[car_type] = {
            'avg_soc': avg_soc,
            'avg_soh': avg_soh,
            'device_count': device_count
        }

    return avg_data

In [ ]:
# 실행
avg_data = get_avg_soc_soh_by_cartype_webapi_style()

# DataFrame 변환
df_avg = pd.DataFrame([
    {"차종": k, "평균_SOC(%)": v["avg_soc"], "평균_SOH(%)": v["avg_soh"], "디바이스수": v["device_count"]}
    for k, v in avg_data.items()
])

print("\n📊 차종별 평균 SOC / SOH 요약:")
print(df_avg.to_string(index=False))

from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'xy'}, {'type': 'xy'}]],
    subplot_titles=('차종별 평균 SOC(%)', '차종별 평균 SOH(%)'),
    horizontal_spacing=0.12
)

# SOC
fig.add_trace(
    go.Bar(x=df_avg['차종'], y=df_avg['평균_SOC(%)'],
           marker_color='#4F6DFF',
           text=df_avg['평균_SOC(%)'], texttemplate='%{text:.2f}%'),
    row=1, col=1
)

# SOH
fig.add_trace(
    go.Bar(x=df_avg['차종'], y=df_avg['평균_SOH(%)'],
           marker_color='#00C2A8',
           text=df_avg['평균_SOH(%)'], texttemplate='%{text:.2f}%'),
    row=1, col=2
)

# 두 서브플롯 모두 동일 축(0~100)
fig.update_yaxes(range=[0, 100], dtick=10, title_text='%', row=1, col=1)
fig.update_yaxes(range=[0, 100], dtick=10, title_text='%', row=1, col=2)

# 2:1 비율
fig.update_layout(title='차종별 평균 SOC/SOH(%)', showlegend=False, width=1000, height=500)
fig.show()

##### 주행 거리
- 컬럼: BMS odometer (주행거리계)
- 계산: odometer 최대값 - 최소값 = 누적 주행거리
- 구현: 차종별 odometer min/max 조회 후 차이 계산

In [17]:
def get_driving_distance_by_cartype():
    """차종별 주행 거리 계산 (odometer 증가분 합산 + 리셋 이벤트 처리)"""
    car_types = ['BONGO3', 'GV60', 'PORTER2']
    driving_distances = {}

    RESET_THRESHOLD = -50  # km 단위, 이 값보다 크게 음수면 리셋 이벤트로 간주
    NOISE_THRESHOLD = -1   # km 단위, 이 값보다 작으면 노이즈로 무시

    for car_type in car_types:
        query = f'''
        from(bucket: "{bucket_name}")
        |> range(start: {min_time}, stop: {max_time})
        |> filter(fn: (r) => r._measurement == "aicar_bms")
        |> filter(fn: (r) => r._field == "odometer")
        |> filter(fn: (r) => r.car_type == "{car_type}")
        |> filter(fn: (r) => exists r._value)
        |> sort(columns: ["_time"])
        '''

        try:
            result = query_api.query(query)
            values, times = [], []
            for table in result:
                for record in table.records:
                    values.append(auto_convert(record.get_value()))
                    times.append(record.get_time())

            total_distance, daily_distances = 0, {}
            sessions, current_session = [], 0

            if values:
                for i in range(1, len(values)):
                    diff = values[i] - values[i-1]

                    if diff > 0:  # 정상 주행
                        total_distance += diff
                        current_session += diff
                        day = times[i].date()
                        daily_distances[day] = daily_distances.get(day, 0) + diff

                    elif diff <= NOISE_THRESHOLD and diff > RESET_THRESHOLD:
                        # 작은 음수 → 노이즈 무시 (=odometer 값이 잠깐 잘못 찍혀도 거리 합산에 반영x)
                        continue

                    elif diff <= RESET_THRESHOLD:
                        # 큰 음수 → 리셋 이벤트, 세션 종료 (=거리 합산 포함x, 새로운 주행 세션 기준으로만 사용)
                        sessions.append(current_session)
                        current_session = 0

                # 마지막 세션 저장
                if current_session > 0:
                    sessions.append(current_session)

            driving_distances[car_type] = {
                'total_distance': total_distance,
                'sessions': sessions,
                'min_odometer': min(values) if values else 0,
                'max_odometer': max(values) if values else 0,
                'daily': daily_distances
            }
            print(f"✅ {car_type}: {total_distance:,.1f}km, 세션 {len(sessions)}개")

        except Exception as e:
            print(f"❌ {car_type}: 주행 거리 계산 실패 - {e}")
            driving_distances[car_type] = {
                'total_distance': 0, 'sessions': [], 'min_odometer': 0, 'max_odometer': 0, 'daily': {}
            }

    return driving_distances

In [18]:
# 실행
driving_distances = get_driving_distance_by_cartype()

# 1. 도넛 차트 (차종별 총 주행거리 비율)
df_total = pd.DataFrame([
    {"차종": k, "총_주행거리": v["total_distance"]} for k,v in driving_distances.items()
])
fig1 = go.Figure(data=[go.Pie(
    labels=df_total["차종"],
    values=df_total["총_주행거리"],
    hole=0.4,
    textinfo="label+percent+value"
)])
fig1.update_layout(title="차종별 총 주행거리 비율 (도넛 차트)")
fig1.show()

# 2. 라인 차트 (일별 주행거리 추이)
df_daily = []
for car_type, data in driving_distances.items():
    for day, dist in data['daily'].items():
        df_daily.append({"날짜": day, "차종": car_type, "주행거리": dist})
df_daily = pd.DataFrame(df_daily)

fig2 = px.line(df_daily, x="날짜", y="주행거리", color="차종",
               title="차종별 일별 주행거리 추이", markers=True)
fig2.update_layout(yaxis_title="주행거리 (km)")
fig2.show()

# 3. 박스플롯 (주행거리 분포)
fig3 = px.box(df_daily, x="차종", y="주행거리", points="all",
              title="차종별 일별 주행거리 분포")
fig3.update_layout(yaxis_title="주행거리 (km)")
fig3.show()

✅ BONGO3: 1,906,698.0km, 세션 142개
✅ GV60: 48,048.0km, 세션 4개
✅ PORTER2: 1,710,944.0km, 세션 39개


- 개별 차량별 주행거리

In [19]:
def get_driving_distance_by_device(device_no: str):
    """개별 차량(device_no) 주행 거리 계산 (odometer 증가분 합산 + 리셋 이벤트 처리)"""
    RESET_THRESHOLD = -50   # km
    NOISE_THRESHOLD  = -1   # km

    query = f'''
    from(bucket: "{bucket_name}")
      |> range(start: {min_time}, stop: {max_time})
      |> filter(fn: (r) => r._measurement == "aicar_bms")
      |> filter(fn: (r) => r._field == "odometer")
      |> filter(fn: (r) => r.device_no == "{device_no}")
      |> filter(fn: (r) => exists r._value)
      |> sort(columns: ["_time"])
    '''

    try:
        result = query_api.query(query)
        odometer_values, timestamps = [], []
        for table in result:
            for record in table.records:
                odometer_values.append(auto_convert(record.get_value()))
                timestamps.append(record.get_time())

        total_distance = 0.0
        daily_distances = {}
        sessions = []
        current_session = 0.0

        if odometer_values:
            for i in range(1, len(odometer_values)):
                diff = odometer_values[i] - odometer_values[i - 1]

                if diff > 0:
                    total_distance += diff
                    current_session += diff
                    day = timestamps[i].date()
                    daily_distances[day] = daily_distances.get(day, 0.0) + diff
                elif NOISE_THRESHOLD >= diff > RESET_THRESHOLD:
                    # 소음: 무시
                    continue
                elif diff <= RESET_THRESHOLD:
                    # 리셋 이벤트: 세션 종료
                    sessions.append(current_session)
                    current_session = 0.0

            if current_session > 0:
                sessions.append(current_session)

        result_obj = {
            "device_no": device_no,
            "total_distance": total_distance,
            "sessions": sessions,
            "min_odometer": min(odometer_values) if odometer_values else 0.0,
            "max_odometer": max(odometer_values) if odometer_values else 0.0,
            "daily": daily_distances,
        }
        print(f"✅ {device_no}: {total_distance:,.1f}km, 세션 {len(sessions)}개")
        return result_obj

    except Exception as e:
        print(f"❌ {device_no}: 주행 거리 계산 실패 - {e}")
        return {
            "device_no": device_no,
            "total_distance": 0.0,
            "sessions": [],
            "min_odometer": 0.0,
            "max_odometer": 0.0,
            "daily": {},
        }

In [20]:
def fetch_device_nos(car_type: str | None = None) -> list[str]:
    # device_no 목록 동적 조회
    car_type_filter = f'|> filter(fn: (r) => r.car_type == "{car_type}")' if car_type else ""
    query = f'''
    from(bucket: "{bucket_name}")
      |> range(start: {min_time}, stop: {max_time})
      |> filter(fn: (r) => r._measurement == "aicar_bms")
      {car_type_filter}
      |> keep(columns: ["device_no"])
      |> distinct(column: "device_no")
    '''
    result = query_api.query(query)
    devices = set()
    for table in result:
        for record in table.records:
            dn = record.values.get("device_no")
            if dn:
                devices.add(str(dn))
    return sorted(devices)

In [ ]:
# device_no 목록 동적 수집 (차종 필터 없으면 전체)
device_list = fetch_device_nos(car_type=None)  # 예: car_type="GV60" 로 특정 차종만
print(f"조회된 device_no 수: {len(device_list)}")

# 필요 시 상한
# device_list = device_list[:50]

# per-device 계산
results = [get_driving_distance_by_device(d) for d in device_list]

# 총 주행거리 비교 차트
# df_total = pd.DataFrame([{"device_no": r["device_no"], "총_주행거리": r["total_distance"]} for r in results])
# if not df_total.empty:
#     fig1 = go.Figure(data=[go.Pie(labels=df_total["device_no"], values=df_total["총_주행거리"], hole=0.4,
#                                   textinfo="label+percent+value")])
#     fig1.update_layout(title="device_no별 총 주행거리 비율")
#     fig1.show()

# 일별 추이/분포
df_daily = []
for r in results:
    for day, dist_km in r["daily"].items():
        df_daily.append({"날짜": day, "device_no": r["device_no"], "주행거리": dist_km})
df_daily = pd.DataFrame(df_daily)

if not df_daily.empty:
    fig2 = px.line(df_daily, x="날짜", y="주행거리", color="device_no",
                   title="device_no별 일별 주행거리 추이", markers=True)
    fig2.update_layout(yaxis_title="주행거리 (km)")
    fig2.show()

    fig3 = px.box(df_daily, x="device_no", y="주행거리", points="all",
                  title="device_no별 일별 주행거리 분포")
    fig3.update_layout(yaxis_title="주행거리 (km)")
    fig3.show()
else:
    print("일별 주행 데이터가 없습니다.")

조회된 device_no 수: 117
✅ 1229665632: 21,658.0km, 세션 1개
✅ 1241055568: 13,758.0km, 세션 1개
✅ 1241124027: 21,478.0km, 세션 1개
✅ 1241124041: 1,013.0km, 세션 1개
✅ 1241124042: 6,671.0km, 세션 1개
✅ 1241124043: 7,009.0km, 세션 1개
✅ 1241124047: 8,452.0km, 세션 1개
✅ 1241124051: 24,109.0km, 세션 1개
✅ 1241124068: 13,236.0km, 세션 1개
✅ 1241124074: 12,389.0km, 세션 1개
✅ 1241144620: 6,939.0km, 세션 1개
✅ 1241144622: 21,239.0km, 세션 1개
✅ 1241144640: 10,699.0km, 세션 1개
✅ 1241144663: 8,732.0km, 세션 1개
✅ 1241172640: 15,265.0km, 세션 1개
✅ 1241172641: 5,880.0km, 세션 1개
✅ 1241172642: 9,298.0km, 세션 1개
✅ 1241172644: 10,164.0km, 세션 1개
✅ 1241172646: 6,880.0km, 세션 1개
✅ 1241172647: 16,737.0km, 세션 1개
✅ 1241172648: 24,785.0km, 세션 1개
✅ 1241172649: 4,654.0km, 세션 1개
✅ 1241172656: 18,532.0km, 세션 1개
✅ 1241172671: 526.0km, 세션 1개
✅ 1241172689: 38,529.0km, 세션 1개
✅ 1241172690: 34,427.0km, 세션 1개
✅ 1241225137: 9,706.0km, 세션 1개
✅ 1241225142: 6,371.0km, 세션 1개
✅ 1241225154: 6,679.0km, 세션 1개
✅ 1241225158: 25,036.0km, 세션 1개
✅ 1241225159: 42,188.0km, 세션 1개
✅ 1

##### 총 주행 시간
- 컬럼: BMS time + odometer
- 계산: odometer가 증가하는 구간의 시간 합산
- 구현: 주행 구간별 시간 차이 누적

In [10]:
# 총 주행 시간 계산
def get_driving_time_by_cartype():
    """차종별 총 주행 시간 계산 (odometer 증가 구간의 시간 합산)"""
    car_types = ['BONGO3', 'GV60', 'PORTER2']
    driving_times = {}
    
    for car_type in car_types:
        # odometer와 time 데이터 조회 (시간순 정렬)
        query = f'''
        from(bucket: "{bucket_name}")
          |> range(start: {min_time}, stop: {max_time})
          |> filter(fn: (r) => r._measurement == "aicar_bms")
          |> filter(fn: (r) => r._field == "odometer")
          |> filter(fn: (r) => r.car_type == "{car_type}")
          |> filter(fn: (r) => exists r._value)
          |> sort(columns: ["_time"])
        '''
        
        try:
            result = query_api.query(query)
            timestamps = []
            odometer_values = []
            
            for table in result:
                for record in table.records:
                    timestamps.append(record.get_time())
                    odometer_values.append(record.values.get('_value', 0))
            
            # 주행 시간 계산 (odometer가 증가하는 구간만)
            total_driving_time = 0
            if len(timestamps) > 1:
                for i in range(1, len(timestamps)):
                    # odometer가 증가한 경우만 주행으로 간주
                    if odometer_values[i] > odometer_values[i-1]:
                        time_diff = (timestamps[i] - timestamps[i-1]).total_seconds()
                        total_driving_time += time_diff
            
            # 시간을 시간:분:초 형태로 변환
            hours = int(total_driving_time // 3600)
            minutes = int((total_driving_time % 3600) // 60)
            seconds = int(total_driving_time % 60)
            
            driving_times[car_type] = {
                'total_seconds': total_driving_time,
                'hours': hours,
                'minutes': minutes,
                'seconds': seconds,
                'formatted_time': f"{hours:02d}:{minutes:02d}:{seconds:02d}"
            }
            print(f"✅ {car_type}: {hours:02d}:{minutes:02d}:{seconds:02d} ({total_driving_time:,.0f}초)")
            
        except Exception as e:
            print(f"❌ {car_type}: 주행 시간 계산 실패 - {e}")
            driving_times[car_type] = {
                'total_seconds': 0,
                'hours': 0,
                'minutes': 0,
                'seconds': 0,
                'formatted_time': "00:00:00"
            }
    
    return driving_times

In [11]:
# 총 주행 시간 계산 실행
driving_times = get_driving_time_by_cartype()

# 결과를 DataFrame으로 변환
time_data = []
for car_type, data in driving_times.items():
    time_data.append({
        '차종': car_type,
        '총_주행시간(초)': data['total_seconds'],
        '시간': data['hours'],
        '분': data['minutes'],
        '초': data['seconds'],
        '포맷된_시간': data['formatted_time']
    })

df_time = pd.DataFrame(time_data)

# 막대 그래프
fig1 = px.bar(df_time, x='차종', y='시간', 
              title='차종별 총 주행 시간',
              color='차종',
              text='포맷된_시간')

fig1.update_traces(textposition='outside')
fig1.update_layout(
    yaxis_title='주행 시간 (시간)',
    height=500,
    showlegend=False
)
fig1.show()

# 히트맵 (시간대별)
fig2 = go.Figure(data=go.Heatmap(
    z=df_time[['시간', '분', '초']].values,
    x=['시간', '분', '초'],
    y=df_time['차종'],
    text=df_time[['시간', '분', '초']].values,
    texttemplate="%{text}",
    textfont={"size": 16},
    colorscale='Viridis'
))

fig2.update_layout(
    title='차종별 주행 시간 구성 (히트맵)',
    xaxis_title='시간 단위',
    yaxis_title='차종',
    height=400
)
fig2.show()

# 테이블로 표시
print("\n📊 차종별 총 주행 시간 요약:")
print(df_time[['차종', '포맷된_시간', '총_주행시간(초)']].to_string(index=False))

# 총합 계산
total_seconds = df_time['총_주행시간(초)'].sum()
total_hours = int(total_seconds // 3600)
total_minutes = int((total_seconds % 3600) // 60)
total_secs = int(total_seconds % 60)
print(f"\n🚗 전체 총 주행 시간: {total_hours:02d}:{total_minutes:02d}:{total_secs:02d} ({total_seconds:,.0f}초)")

KeyboardInterrupt: 

##### 데이터 저장 기간
- 컬럼: BMS time
- 계산: min(time) ~ max(time)
- 구현: 차종별/전체 데이터 기간 분석

In [ ]:
# 데이터 저장 기간 계산 (BMS, GPS 따로)
def get_data_storage_period_by_cartype():
    """차종별 데이터 저장 기간 계산 (BMS, GPS 따로)"""
    car_types = ['BONGO3', 'GV60', 'PORTER2']
    storage_periods = {}
    
    for car_type in car_types:
        # BMS time 최소값과 최대값 조회
        bms_query = f'''
        from(bucket: "{bucket_name}")
          |> range(start: {min_time}, stop: {max_time})
          |> filter(fn: (r) => r._measurement == "aicar_bms")
          |> filter(fn: (r) => r.car_type == "{car_type}")
          |> filter(fn: (r) => exists r._time)
          |> aggregateWindow(every: inf, fn: min, createEmpty: false)
        '''
        
        # GPS time 최소값과 최대값 조회
        gps_query = f'''
        from(bucket: "{bucket_name}")
          |> range(start: {min_time}, stop: {max_time})
          |> filter(fn: (r) => r._measurement == "aicar_gps")
          |> filter(fn: (r) => r.car_type == "{car_type}")
          |> filter(fn: (r) => exists r._time)
          |> aggregateWindow(every: inf, fn: min, createEmpty: false)
        '''
        
        bms_min_time = None
        bms_max_time = None
        gps_min_time = None
        gps_max_time = None
        
        try:
            # BMS 최소 시간 조회
            result = query_api.query(bms_query.replace("fn: min", "fn: min"))
            for table in result:
                for record in table.records:
                    bms_min_time = record.get_time()
                    break
            
            # BMS 최대 시간 조회
            result = query_api.query(bms_query.replace("fn: min", "fn: max"))
            for table in result:
                for record in table.records:
                    bms_max_time = record.get_time()
                    break
            
            # GPS 최소 시간 조회
            result = query_api.query(gps_query.replace("fn: min", "fn: min"))
            for table in result:
                for record in table.records:
                    gps_min_time = record.get_time()
                    break
            
            # GPS 최대 시간 조회
            result = query_api.query(gps_query.replace("fn: min", "fn: max"))
            for table in result:
                for record in table.records:
                    gps_max_time = record.get_time()
                    break
            
            # 전체 기간 계산 (BMS와 GPS 중 가장 빠른 시작, 가장 늦은 종료)
            all_times = [t for t in [bms_min_time, bms_max_time, gps_min_time, gps_max_time] if t is not None]
            
            if all_times:
                overall_start = min(all_times)
                overall_end = max(all_times)
                duration = overall_end - overall_start
                days = duration.days
                hours = duration.seconds // 3600
                minutes = (duration.seconds % 3600) // 60
                
                storage_periods[car_type] = {
                    'bms_start': bms_min_time,
                    'bms_end': bms_max_time,
                    'gps_start': gps_min_time,
                    'gps_end': gps_max_time,
                    'overall_start': overall_start,
                    'overall_end': overall_end,
                    'duration_days': days,
                    'duration_hours': hours,
                    'duration_minutes': minutes,
                    'total_days': days + (hours / 24) + (minutes / 1440)
                }
                print(f"✅ {car_type}: {overall_start.strftime('%Y-%m-%d %H:%M')} ~ {overall_end.strftime('%Y-%m-%d %H:%M')} ({days}일 {hours}시간 {minutes}분)")
            else:
                storage_periods[car_type] = {
                    'bms_start': None, 'bms_end': None,
                    'gps_start': None, 'gps_end': None,
                    'overall_start': None, 'overall_end': None,
                    'duration_days': 0, 'duration_hours': 0, 'duration_minutes': 0, 'total_days': 0
                }
                print(f"❌ {car_type}: 데이터 없음")
            
        except Exception as e:
            print(f"❌ {car_type}: 저장 기간 계산 실패 - {e}")
            storage_periods[car_type] = {
                'bms_start': None, 'bms_end': None,
                'gps_start': None, 'gps_end': None,
                'overall_start': None, 'overall_end': None,
                'duration_days': 0, 'duration_hours': 0, 'duration_minutes': 0, 'total_days': 0
            }
    
    return storage_periods

In [ ]:
# 데이터 저장 기간 계산 실행
storage_periods = get_data_storage_period_by_cartype()

# 결과를 DataFrame으로 변환
period_data = []
for car_type, data in storage_periods.items():
    period_data.append({
        '차종': car_type,
        'BMS_시작': data['bms_start'].strftime('%Y-%m-%d') if data['bms_start'] else 'N/A',
        'BMS_종료': data['bms_end'].strftime('%Y-%m-%d') if data['bms_end'] else 'N/A',
        'GPS_시작': data['gps_start'].strftime('%Y-%m-%d') if data['gps_start'] else 'N/A',
        'GPS_종료': data['gps_end'].strftime('%Y-%m-%d') if data['gps_end'] else 'N/A',
        '전체_시작': data['overall_start'].strftime('%Y-%m-%d') if data['overall_start'] else 'N/A',
        '전체_종료': data['overall_end'].strftime('%Y-%m-%d') if data['overall_end'] else 'N/A',
        '저장기간(일)': data['duration_days'],
        '총_일수': data['total_days']
    })

df_period = pd.DataFrame(period_data)

# 타임라인 (BMS, GPS 따로)
fig1 = go.Figure()

for i, row in df_period.iterrows():
    if row['BMS_시작'] != 'N/A' and row['BMS_종료'] != 'N/A':
        fig1.add_trace(go.Scatter(
            x=[row['BMS_시작'], row['BMS_종료']],
            y=[f"{row['차종']}_BMS", f"{row['차종']}_BMS"],
            mode='lines+markers',
            name=f"{row['차종']}_BMS",
            line=dict(width=6, color='blue'),
            marker=dict(size=10)
        ))
    
    if row['GPS_시작'] != 'N/A' and row['GPS_종료'] != 'N/A':
        fig1.add_trace(go.Scatter(
            x=[row['GPS_시작'], row['GPS_종료']],
            y=[f"{row['차종']}_GPS", f"{row['차종']}_GPS"],
            mode='lines+markers',
            name=f"{row['차종']}_GPS",
            line=dict(width=6, color='red'),
            marker=dict(size=10)
        ))

fig1.update_layout(
    title='차종별 데이터 저장 기간 (BMS vs GPS)',
    xaxis_title='날짜',
    yaxis_title='차종_데이터타입',
    height=500
)
fig1.show()

# 영역 차트
fig2 = go.Figure()

for i, row in df_period.iterrows():
    if row['총_일수'] > 0:
        fig2.add_trace(go.Scatter(
            x=[row['차종']],
            y=[row['총_일수']],
            mode='markers',
            name=row['차종'],
            marker=dict(size=row['총_일수']*2, color=f'rgba({i*100}, {i*150}, {i*200}, 0.6)'),
            fill='tonexty'
        ))

fig2.update_layout(
    title='차종별 데이터 저장 기간 (영역 차트)',
    xaxis_title='차종',
    yaxis_title='저장 기간 (일)',
    height=500
)
fig2.show()

# 테이블로 표시
print("\n📊 차종별 데이터 저장 기간 요약:")
print(df_period.to_string(index=False))

##### 충전 기간 분석 (다중 신호 기반)
- 컬럼: BMS soc, pack_current, chrg_cable_conn, fast_chrg_port_conn, slow_chrg_port_conn
- 계산: SOC 증가 + 양의 전류 + 케이블 연결 + 충전 포트 연결 (최소 3개 신호 일치)
- 구현: 다중 신호 검증으로 오탐지 최소화, 고속충전/완속충전 구분, 신뢰도 점수 계산

In [2]:
def get_charging_analysis_precise():
    """정확한 충전기간 분석 (실제 컬럼 기반)"""
    car_types = ['BONGO3', 'GV60', 'PORTER2']
    charging_data = {}

    for car_type in car_types:
        print(f"🔋 {car_type} 정확한 충전기간 분석 시작...")
        
        # 충전 관련 모든 신호 조회
        charging_query = f'''
        from(bucket: "{bucket_name}")
          |> range(start: {min_time}, stop: {max_time})
          |> filter(fn: (r) => r._measurement == "aicar_bms")
          |> filter(fn: (r) => r._field == "soc" or r._field == "pack_current" or r._field == "pack_volt" or r._field == "chrg_cable_conn" or r._field == "fast_chrg_port_conn" or r._field == "slow_chrg_port_conn")
          |> filter(fn: (r) => r.car_type == "{car_type}")
          |> filter(fn: (r) => exists r._value)
          |> sort(columns: ["_time"])
        '''

        try:
            rows = run_query(charging_query)
            
            # 디바이스별로 데이터 정리
            device_data = {}
            for row in rows:
                device_no = row.get('device_no')
                field = row.get('_field')
                value = row.get('_value')
                time = row.get('_time')
                
                if not device_no:
                    continue
                    
                try:
                    val = float(value)
                except:
                    continue
                
                if device_no not in device_data:
                    device_data[device_no] = []
                
                device_data[device_no].append({
                    'time': time,
                    'field': field,
                    'value': val
                })

            # 각 디바이스별 충전 세션 분석
            total_sessions = 0
            total_charging_time = 0
            charging_sessions = []

            for device_no, data in device_data.items():
                # 시간순 정렬
                data.sort(key=lambda x: x['time'])
                
                # 필드별 데이터 분리
                soc_data = [d for d in data if d['field'] == 'soc']
                current_data = [d for d in data if d['field'] == 'pack_current']
                cable_data = [d for d in data if d['field'] == 'chrg_cable_conn']
                fast_port_data = [d for d in data if d['field'] == 'fast_chrg_port_conn']
                slow_port_data = [d for d in data if d['field'] == 'slow_chrg_port_conn']
                
                # 정확한 충전 세션 감지
                CHARGING_CURRENT_THRESHOLD = 5.0  # A
                SOC_INCREASE_THRESHOLD = 0.1  # %
                MIN_CHARGING_DURATION = 2  # 분
                IDLE_THRESHOLD = 1.0  # A
                
                current_session = None
                
                for i in range(1, len(soc_data)):
                    prev_soc = soc_data[i-1]['value']
                    curr_soc = soc_data[i]['value']
                    time = soc_data[i]['time']
                    
                    # 해당 시간대의 모든 신호 찾기
                    current_at_time = None
                    cable_connected = False
                    fast_port_connected = False
                    slow_port_connected = False
                    
                    for curr in current_data:
                        if abs((curr['time'] - time).total_seconds()) < 60:
                            current_at_time = curr['value']
                            break
                    
                    for cable in cable_data:
                        if abs((cable['time'] - time).total_seconds()) < 60:
                            cable_connected = cable['value'] > 0
                            break
                    
                    for fast in fast_port_data:
                        if abs((fast['time'] - time).total_seconds()) < 60:
                            fast_port_connected = fast['value'] > 0
                            break
                    
                    for slow in slow_port_data:
                        if abs((slow['time'] - time).total_seconds()) < 60:
                            slow_port_connected = slow['value'] > 0
                            break
                    
                    # 충전 시작 감지 (다중 신호 확인)
                    charging_signals = [
                        curr_soc > prev_soc + SOC_INCREASE_THRESHOLD,  # SOC 증가
                        current_at_time and current_at_time > CHARGING_CURRENT_THRESHOLD,  # 양의 전류
                        cable_connected,  # 케이블 연결
                        fast_port_connected or slow_port_connected  # 충전 포트 연결
                    ]
                    
                    # 최소 3개 이상의 신호가 일치해야 충전으로 인식
                    if sum(charging_signals) >= 3 and not current_session:
                        charging_type = "고속충전" if fast_port_connected else "완속충전" if slow_port_connected else "충전"
                        
                        current_session = {
                            'start_time': soc_data[i-1]['time'],
                            'end_time': time,
                            'start_soc': prev_soc,
                            'end_soc': curr_soc,
                            'soc_increase': curr_soc - prev_soc,
                            'start_current': current_at_time,
                            'charging_type': charging_type,
                            'cable_connected': cable_connected,
                            'fast_port_connected': fast_port_connected,
                            'slow_port_connected': slow_port_connected,
                            'duration_minutes': 0,
                            'device_no': device_no,
                            'confidence': sum(charging_signals) / len(charging_signals)  # 신뢰도
                        }
                    
                    # 충전 중 (SOC 증가 + 양의 전류 지속)
                    elif (current_session and 
                          curr_soc > prev_soc and 
                          current_at_time and current_at_time > IDLE_THRESHOLD):
                        
                        current_session['end_time'] = time
                        current_session['end_soc'] = curr_soc
                        current_session['soc_increase'] = curr_soc - current_session['start_soc']
                        
                        # 시간 계산
                        time_diff = (time - current_session['start_time']).total_seconds() / 60
                        current_session['duration_minutes'] = time_diff
                    
                    # 충전 종료 감지
                    elif (current_session and 
                          (curr_soc <= prev_soc or 
                           (current_at_time and current_at_time <= IDLE_THRESHOLD) or
                           not cable_connected)):
                        
                        # 최소 충전 시간 확인
                        if current_session['duration_minutes'] >= MIN_CHARGING_DURATION:
                            charging_sessions.append(current_session)
                            total_sessions += 1
                            total_charging_time += current_session['duration_minutes']
                        
                        current_session = None

            # 통계 계산
            avg_session_duration = total_charging_time / total_sessions if total_sessions > 0 else 0
            fast_charging_sessions = len([s for s in charging_sessions if s['charging_type'] == '고속충전'])
            slow_charging_sessions = len([s for s in charging_sessions if s['charging_type'] == '완속충전'])
            
            charging_data[car_type] = {
                'total_sessions': total_sessions,
                'total_charging_time_hours': total_charging_time / 60,
                'avg_session_duration_minutes': avg_session_duration,
                'fast_charging_sessions': fast_charging_sessions,
                'slow_charging_sessions': slow_charging_sessions,
                'sessions': charging_sessions[:10]
            }
            
            print(f"✅ {car_type}: 충전 세션 {total_sessions}개 (고속 {fast_charging_sessions}, 완속 {slow_charging_sessions})")

        except Exception as e:
            print(f"❌ {car_type}: 충전기간 분석 실패 - {e}")

    return charging_data

In [ ]:
# 충전 기간 데이터 분석 실행
charging_data = get_charging_data_by_cartype()

# 결과를 DataFrame으로 변환
charging_stats = []
for car_type, data in charging_data.items():
    charging_stats.append({
        '차종': car_type,
        '충전횟수': data['total_sessions'],
        '총충전시간(초)': data['total_duration'],
        '총충전시간(시간)': data['total_duration'] / 3600,
        '평균충전시간(초)': data['avg_duration'],
        '평균충전시간(시간)': data['avg_duration'] / 3600
    })

df_charging = pd.DataFrame(charging_stats)

# 라인 차트 (시간별 충전 패턴)
fig1 = go.Figure()

for car_type, data in charging_data.items():
    if data['sessions']:
        times = [session['start_time'] for session in data['sessions']]
        durations = [session['duration'] / 3600 for session in data['sessions']]  # 시간 단위로 변환
        
        fig1.add_trace(go.Scatter(
            x=times,
            y=durations,
            mode='lines+markers',
            name=car_type,
            line=dict(width=2),
            marker=dict(size=6)
        ))

fig1.update_layout(
    title='차종별 충전 시간 패턴 (라인 차트)',
    xaxis_title='시간',
    yaxis_title='충전 시간 (시간)',
    height=500
)
fig1.show()

# 박스플롯 (충전 시간 분포)
fig2 = go.Figure()

for car_type, data in charging_data.items():
    if data['sessions']:
        durations = [session['duration'] / 3600 for session in data['sessions']]  # 시간 단위로 변환
        
        fig2.add_trace(go.Box(
            y=durations,
            name=car_type,
            boxpoints='all',
            jitter=0.3,
            pointpos=-1.8
        ))

fig2.update_layout(
    title='차종별 충전 시간 분포 (박스플롯)',
    yaxis_title='충전 시간 (시간)',
    height=500
)
fig2.show()

# 테이블로 표시
print("\n📊 차종별 충전 기간 데이터 요약:")
print(df_charging.to_string(index=False))

# 전체 통계
total_sessions = df_charging['충전횟수'].sum()
total_duration = df_charging['총충전시간(시간)'].sum()
print(f"\n🔋 전체 충전 횟수: {total_sessions}회")
print(f"🔋 전체 충전 시간: {total_duration:.1f}시간")

##### 머신러닝 기반 분석
- 데이터: BMS - soc, soh, pack_current, pack_volt, batt_internal_temp, mod_temp_1~18, cumul_energy_chrgd
- 분석 예정:
    - 충전 효율 예측 (SOC 변화율 + 충전 에너지)
    - 배터리 수명 예측 (SOH + 충전 패턴 + 온도)
    - 주행 패턴 분류 (주행거리 + 충전 패턴)
    - 이상 충전 감지 (충전 패턴 이상 탐지)